Натренировать классификатор сентимента твитов на основе корпуса и получить confusion matrix. Дополнительно нужно посмотреть, как зависит качество от разной токенизации при векторизации.

http://study.mokoron.com/
    
База данных состоит из 12 столбцов:
– id: уникальный номер сообщения в системе twitter;
– tdate: дата публикации сообщения (твита);
– tmane: имя пользователя, опубликовавшего сообщение;
– ttext:  текст сообщения (твита);
– ttype: поле в котором в дальнейшем будет указано к кому классу относится твит (положительный(1), отрицательный(-1), нейтральный(0));
– trep: количество реплаев к данному сообщению. В настоящий момент API твиттера не отдает эту информацию;
– tfav: число сколько раз данное сообщение было добавлено в избранное другими пользователями;
– tstcount: число всех сообщений пользователя в сети twitter;
– tfol: количество фоловеров пользователя (тех людей, которые читают пользователя);
– tfrien: количество друзей пользователя (те люди, которых читает пользователь);
– listcount: количество листов-подписок в которые добавлен твиттер-пользователь.

In [87]:
import re
import collections
import pandas as pd
import numpy as np
import sklearn

In [88]:
settings = {
    'sep':';',
    'header':None,
    'names':['id','tdate','tname','ttext','ttype','trep', 'trtw', 'tfav','tstcount','tfol','tfrien','listcount'],
    'index_col':'id',
    'parse_dates':[1],
    'date_parser':lambda x: pd.to_datetime(x, unit='s'),
}


positive = pd.read_csv('positive.csv', **settings)
negative = pd.read_csv('negative.csv', **settings)
corpus = pd.concat([positive, negative])

## Pre-processing 

In [89]:
corpus.head()

,tdate,tname,ttext,ttype,trep,trtw,tfav,tstcount,tfol,tfrien,listcount
id,,,,,,,,,,,
408906692374446080,2013-12-06 10:32:07,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
408906692693221377,2013-12-06 10:32:07,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
408906695083954177,2013-12-06 10:32:07,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
408906695356973056,2013-12-06 10:32:07,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
408906761416867842,2013-12-06 10:32:23,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1


In [90]:
negative.head(1)

,tdate,tname,ttext,ttype,trep,trtw,tfav,tstcount,tfol,tfrien,listcount
id,,,,,,,,,,,
408906762813579264,2013-12-06 10:32:24,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2


In [91]:
positive.head(1)

,tdate,tname,ttext,ttype,trep,trtw,tfav,tstcount,tfol,tfrien,listcount
id,,,,,,,,,,,
408906692374446080,2013-12-06 10:32:07,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0


In [92]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226834 entries, 408906692374446080 to 425138595251625984
Data columns (total 11 columns):
tdate        226834 non-null datetime64[ns]
tname        226834 non-null object
ttext        226834 non-null object
ttype        226834 non-null int64
trep         226834 non-null int64
trtw         226834 non-null int64
tfav         226834 non-null int64
tstcount     226834 non-null int64
tfol         226834 non-null int64
tfrien       226834 non-null int64
listcount    226834 non-null int64
dtypes: datetime64[ns](1), int64(8), object(2)
memory usage: 20.8+ MB


In [93]:
corpus_1 = pd.get_dummies(corpus, columns=['ttype'])

In [94]:
X = corpus_1.ttext

In [95]:
y = corpus_1.ttype_1

### Bag of words 

### 3.1. CountVectorizer

In [96]:
import sklearn.feature_extraction.text

In [97]:
vctr = sklearn.feature_extraction.text.CountVectorizer() 

### 3.2. TF.IDF vectorizer

In [98]:
vctr_tfidf = sklearn.feature_extraction.text.TfidfVectorizer()

### 3.3 Classify - Naive model

In [99]:
from sklearn.naive_bayes import MultinomialNB # используеим наив байс в качестве классификатора 
clf1 = MultinomialNB()

### 3.4 Classify - Support Vector Machine

In [100]:
from sklearn.svm import LinearSVC
clf2 = sklearn.svm.LinearSVC()

### Pipeline

In [101]:
# готовим две комбинации pipeline

In [102]:
from sklearn.pipeline import Pipeline

In [103]:
pipeline_1 = Pipeline([
    ('bow', vctr),  # strings to token integer counts
    ('nb', clf1),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [104]:
pipeline_2 = Pipeline([
    ('tfidf', vctr_tfidf),  # integer counts to weighted TF-IDF scores
    ('svm', clf2), #Support vector machines 
])

## Grid search

In [105]:
from time import time
from sklearn.model_selection import GridSearchCV

### Grid search countvectorizer Naive Bayes

In [106]:
parameters_1 = {
    'bow__max_df': (0.9, 0.95, 0.98),
    #'vect__max_features': (None, 5000, 10000, 50000),
    'bow__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    #'tfidf__use_idf': (True, False),
    #'tfidf__norm': ('l1', 'l2'),
    #'clf__alpha': (0.00001, 0.000001),
    #'clf__penalty': ('l2', 'elasticnet'),
    #'clf__n_iter': (10, 50, 80),
}

In [108]:
grid_search_NB = GridSearchCV(pipeline_1, parameters_1, n_jobs=-1, verbose=1)
grid_search_NB.fit(X,y)
print()
print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search_NB.best_estimator_.get_params()
for param_name in sorted(parameters_1.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:  2.8min finished



Best score: 0.738
Best parameters set:
	bow__max_df: 0.9
	bow__ngram_range: (1, 2)


In [109]:
best_model_1 = grid_search_NB.best_estimator_  # лучшая комбинация - модель pipeleine

### Predictions and Evaluations - Naive Baise

In [110]:
from sklearn.metrics import confusion_matrix,classification_report

In [111]:
predictions_1 = best_model_1.predict(X)

In [112]:
print(confusion_matrix(y,predictions_1))
print('\n')
print(classification_report(y,predictions_1))

[[110595   1328]
 [  4381 110530]]


             precision    recall  f1-score   support

          0       0.96      0.99      0.97    111923
          1       0.99      0.96      0.97    114911

avg / total       0.98      0.97      0.97    226834



### Grid search TF-IDF, SVM classify 

In [113]:
parameters_2 = {
    #'bow__max_df': (0.9, 0.95, 0.98),
    #'vect__max_features': (None, 5000, 10000, 50000),
    #'bow__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    #'clf__alpha': (0.00001, 0.000001),
    #'clf__penalty': ('l2', 'elasticnet'),
    #'clf__n_iter': (10, 50, 80),
}

In [114]:
grid_search_SVM = GridSearchCV(pipeline_2, parameters_2, n_jobs=-1, verbose=1)
grid_search_SVM.fit(X,y)
print()
print("Best score: %0.3f" % grid_search_SVM.best_score_)
print("Best parameters set:")
best_parameters = grid_search_SVM.best_estimator_.get_params()
for param_name in sorted(parameters_2.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  1.4min finished



Best score: 0.738
Best parameters set:
	tfidf__norm: 'l1'
	tfidf__use_idf: True


In [115]:
best_model_2 = grid_search_SVM.best_estimator_ 

### Predictions and Evaluations - SVM

In [116]:
predictions_2 = best_model_2.predict(X)

In [117]:
print(confusion_matrix(y,predictions_2))
print('\n')
print(classification_report(y,predictions_2))

[[ 92713  19210]
 [ 14290 100621]]


             precision    recall  f1-score   support

          0       0.87      0.83      0.85    111923
          1       0.84      0.88      0.86    114911

avg / total       0.85      0.85      0.85    226834



In [118]:
grid_search_SVM.cv_results_['mean_test_score']

array([ 0.73793611,  0.7321301 ,  0.73044164,  0.73677227])

In [119]:
grid_search_SVM.best_score_

0.73793611187035457

## Резюме

Наиболее оптимальная модель - это сочетание CountVectorizer (max_df: 0.9, ngram_range: (1, 2)) и классификатор Naive Baise